In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Assume each base model gives a (1,6) output (6-class softmax probabilities)
num_models = 4
num_classes = 6  # Softmax output size from each base model

# Correct input shape: 4 models * 6 outputs = 24 features
input_shape = num_models * num_classes  # 24

from tensorflow.keras.layers import BatchNormalization, Dropout

meta_classifier = Sequential([
    Dense(64, activation='relu', input_shape=(input_shape,)),  
    BatchNormalization(),  # Normalize inputs for faster learning
    Dropout(0.3),  # Reduce overfitting

    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),

    Dense(16, activation='relu'),

    Dense(num_classes, activation='softmax')  # Output Layer (6 pollution categories)
])
from tensorflow.keras.optimizers import AdamW

# AdamW is better than Adam due to decoupled weight decay
optimizer = AdamW(learning_rate=0.001, weight_decay=1e-5)

meta_classifier.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Summary
meta_classifier.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                1600      
                                                                 
 batch_normalization (Batch  (None, 64)                256       
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 batch_normalization_1 (Bat  (None, 32)                128       
 chNormalization)                                                
                                                                 
 dropout_1 (Dropout)         (None, 32)               

In [7]:
# Simulated training data (Replace with actual predictions from base models)
X_train_meta = np.random.rand(1000, 24)  # 1000 samples, 24 features (4 models × 6 softmax outputs)
y_train_meta = np.random.randint(0, 6, size=(1000,))  # True categories

# Convert to one-hot encoding
y_train_meta = tf.keras.utils.to_categorical(y_train_meta, num_classes=6)

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-5)

meta_classifier.fit(X_train_meta, y_train_meta, 
                    epochs=300, batch_size=32, validation_split=0.2, 
                    callbacks=[early_stopping, reduce_lr])

# Save Model
meta_classifier.save("meta_classifier.keras")

print("✅ Meta-Classifier Trained & Saved Successfully!")


Epoch 1/300
25/25 [==============================] - 0s 13ms/step - loss: 1.8646 - accuracy: 0.1562 - val_loss: 1.7939 - val_accuracy: 0.1700 - lr: 6.2500e-05
Epoch 2/300
25/25 [==============================] - 0s 9ms/step - loss: 1.8521 - accuracy: 0.1675 - val_loss: 1.7929 - val_accuracy: 0.1650 - lr: 6.2500e-05
Epoch 3/300
25/25 [==============================] - 0s 9ms/step - loss: 1.8608 - accuracy: 0.1675 - val_loss: 1.7917 - val_accuracy: 0.1700 - lr: 6.2500e-05
Epoch 4/300
25/25 [==============================] - 0s 9ms/step - loss: 1.8603 - accuracy: 0.1625 - val_loss: 1.7914 - val_accuracy: 0.1750 - lr: 6.2500e-05
Epoch 5/300
25/25 [==============================] - 0s 9ms/step - loss: 1.8587 - accuracy: 0.1637 - val_loss: 1.7907 - val_accuracy: 0.1700 - lr: 6.2500e-05
Epoch 6/300
25/25 [==============================] - 0s 9ms/step - loss: 1.8336 - accuracy: 0.1787 - val_loss: 1.7905 - val_accuracy: 0.1700 - lr: 6.2500e-05
Epoch 7/300
25/25 [==============================] 